LASSO (Least Absolute Shrinkage and Selection Operator) é um método de regressão linear que busca melhorar a precisão dos modelos e reduzir a complexidade ao aplicar uma penalidade sobre os coeficientes de regressão, forçando alguns deles a serem exatamente zero. Isso resulta na seleção automática de variáveis, ou seja, o modelo mantém apenas as mais relevantes. O LASSO é especialmente útil quando há muitas variáveis explicativas e se deseja evitar overfitting, ao mesmo tempo em que se simplifica o modelo.

In [ ]:
import shap
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import GridSearchCV

Limpeza dos Dados:

In [ ]:
df = pd.read_csv('./Data/jogadores.csv')

In [ ]:
lista = ['age', 'birthday', 'birthday_GMT', 'league', 'season', 'nationality', 'clean_sheets_overall', 'clean_sheets_home', 'clean_sheets_away', 'conceded_overall', 'conceded_away', 'conceded_home', 'yellow_cards_overall', 'red_cards_overall', 'min_per_conceded_overall', 'min_per_card_overall', 'cards_per_90_overall', 'rank_in_league_top_attackers', 'rank_in_league_top_midfielders', 'rank_in_league_top_defenders', 'rank_in_club_top_scorer', 'passes_per_90_overall', 'passes_per_game_overall', 'passes_per90_percentile_overall', 'passes_total_overall', 'passes_completed_per_game_overall', 'passes_completed_total_overall', 'pass_completion_rate_percentile_overall', 'passes_completed_per_90_overall', 'passes_completed_per90_percentile_overall', 'short_passes_per_game_overall', 'long_passes_per_game_overall', 'key_passes_per_game_overall', 'key_passes_total_overall', 'through_passes_per_game_overall', 'crosses_per_game_overall', 'dispossesed_per_game_overall', 'possession_regained_per_game_overall', 'pressures_per_game_overall', 'saves_per_game_overall', 'interceptions_per_game_overall', 'shots_faced_per_game_overall', 'shots_per_goal_scored_overall', 'shots_off_target_per_game_overall', 'distance_travelled_per_game_overall', 'possession_regained_per_90_overall', 'possession_regained_total_overall', 'possession_regained_per90_percentile_overall', 'additional_info', 'shots_off_target_total_overall', 'shots_off_target_per_90_overall', 'shots_off_target_per90_percentile_overall', 'games_subbed_out', 'interceptions_total_overall', 'interceptions_per_90_overall', 'interceptions_per90_percentile_overall', 'crosses_total_overall', 'cross_completion_rate_percentile_overall', 'crosses_per_90_overall', 'crosses_per90_percentile_overall', 'through_passes_total_overall', 'through_passes_per_90_overall', 'through_passes_per90_percentile_overall', 'long_passes_total_overall', 'long_passes_per_90_overall', 'long_passes_per90_percentile_overall', 'short_passes_total_overall', 'short_passes_per_90_overall', 'short_passes_per90_percentile_overall', 'key_passes_per_90_overall', 'key_passes_per90_percentile_overall', 'dribbles_total_overall', 'dribbles_per_90_overall', 'dribbles_per90_percentile_overall', 'dribbles_successful_total_overall', 'dribbles_successful_per_90_overall', 'dribbles_successful_percentage_overall', 'chances_created_total_overall', 'chances_created_per_90_overall', 'chances_created_per90_percentile_overall', 'saves_total_overall', 'save_percentage_percentile_overall', 'saves_per_90_overall', 'saves_per90_percentile_overall', 'shots_faced_total_overall', 'shots_per_goal_conceded_overall', 'shots_faced_per_90_overall', 'shots_faced_per90_percentile_overall', 'xg_faced_per_90_overall', 'xg_faced_per90_percentile_overall', 'xg_faced_per_game_overall', 'xg_faced_total_overall', 'save_percentage_overall', 'pressures_total_overall', 'pressures_per_90_overall', 'pressures_per90_percentile_overall', 'xg_total_overall', 'market_value', 'market_value_percentile', 'pass_completion_rate_overall', 'dribbled_past_per90_percentile_overall', 'dribbled_past_per_game_overall', 'dribbled_past_per_90_overall', 'dribbled_past_total_overall', 'inside_box_saves_total_overall', 'blocks_per_game_overall', 'blocks_per_90_overall', 'blocks_total_overall', 'blocks_per90_percentile_overall', 'ratings_total_overall', 'clearances_per_game_overall', 'clearances_total_overall', 'clearances_per_90_overall', 'pen_save_percentage_overall', 'pen_committed_total_overall', 'pen_committed_per_90_overall', 'pen_committed_per90_percentile_overall', 'pen_committed_per_game_overall', 'pens_saved_total_overall', 'pens_taken_total_overall', 'hit_woodwork_total_overall', 'hit_woodwork_per_90_overall', 'punches_total_overall', 'punches_per_game_overall', 'punches_per_90_overall', 'offsides_per_90_overall', 'offsides_per_game_overall', 'offsides_total_overall', 'shot_conversion_rate_overall', 'shot_conversion_rate_percentile_overall', 'sm_minutes_played_per90_percentile_overall', 'sm_minutes_played_recorded_overall', 'min_per_goal_percentile_overall', 'min_per_conceded_percentile_overall', 'xa_total_overall', 'xa_per90_percentile_overall', 'xa_per_game_overall', 'xa_per_90_overall', 'npxg_total_overall', 'npxg_per90_percentile_overall', 'npxg_per_game_overall', 'npxg_per_90_overall', 'fouls_drawn_per90_percentile_overall', 'fouls_drawn_total_overall', 'fouls_drawn_per_game_overall', 'fouls_drawn_per_90_overall', 'fouls_committed_per_90_overall', 'fouls_committed_per_game_overall', 'fouls_committed_per90_percentile_overall', 'fouls_committed_total_overall', 'xg_per_90_overall', 'xg_per90_percentile_overall', 'average_rating_percentile_overall', 'clearances_per90_percentile_overall', 'hit_woodwork_per90_percentile_overall', 'punches_per90_percentile_overall', 'offsides_per90_percentile_overall', 'aerial_duels_total_overall', 'aerial_duels_per_90_overall', 'aerial_duels_per90_percentile_overall', 'aerial_duels_won_percentage_overall', 'duels_per_game_overall', 'duels_total_overall', 'duels_won_total_overall', 'duels_won_per90_percentile_overall', 'duels_per90_percentile_overall', 'duels_won_per_90_overall', 'duels_won_per_game_overall', 'duels_won_percentage_overall', 'dispossesed_total_overall', 'dispossesed_per_90_overall', 'dispossesed_per90_percentile_overall', 'progressive_passes_total_overall', 'cross_completion_rate_overall', 'distance_travelled_total_overall', 'distance_travelled_per_90_overall', 'distance_travelled_per90_percentile_overall', 'accurate_crosses_total_overall', 'accurate_crosses_per_game_overall', 'accurate_crosses_per_game_overall', 'accurate_crosses_per_90_overall', 'accurate_crosses_per90_percentile_overall', 'sm_matches_recorded_total_overall', 'games_started_percentile_overall', 'games_subbed_in_percentile_overall', 'games_subbed_out_percentile_overall', 'hattricks_total_overall', 'two_goals_in_a_game_total_overall', 'three_goals_in_a_game_total_overall', 'two_goals_in_a_game_percentage_overall', 'three_goals_in_a_game_percentage_overall', 'man_of_the_match_total_overall', 'annual_salary_eur', 'annual_salary_eur_percentile', 'clean_sheets_percentage_percentile_overall', 'min_per_card_percentile_overall', 'cards_per90_percentile_overall', 'booked_over05_overall', 'booked_over05_percentage_overall', 'booked_over05_percentage_percentile_overall', 'shirt_number', 'annual_salary_gbp', 'annual_salary_usd', 'z_score', 'is_outlier']
# A lista foi criada manualmente, baseando-se em hipóteses de quais features seriam irrelevantes para o modelo.

In [ ]:
new_df = df.drop(columns=lista)
new_df.to_csv('./Data/p.csv')

In [ ]:
df = pd.read_csv('./Data/p.csv')

In [ ]:
non_numeric_cols = df[['full_name', 'position', 'Current Club']].copy()
df_numeric = df.select_dtypes(include=['number']).copy()

In [ ]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
df_numeric_imputed = pd.DataFrame(imp_mean.fit_transform(df_numeric), columns=df_numeric.columns)

In [ ]:
data = pd.concat([non_numeric_cols, df_numeric_imputed], axis=1)

Aplicação do modelo LASSO:

In [ ]:
# Função auxiliar para substituir a coluna de posição
def chance_of_goal(position):
    if position == 'Forward':
        return 1  # Alta chance de marcar gol
    elif position == 'Midfielder':
        return 0.7  # Chance intermediária
    elif position == 'Defender':
        return 0.3 # Baixa chance de marcar gol
    else:
        return 0 # Chance nula

In [ ]:
# Normalizar as previsões para intervalo [0, 1] usando uma transformação sigmoide
def sigmoid(x):
    return np.round((1 / (1 + np.exp(-x))) * 100)

In [ ]:
# Numerando os times para evitar possíveis erros de digitação nas testagens.
times = {
    0: 'Vitória',
    1: 'Flamengo',
    2: 'Cruzeiro',
    3: 'Botafogo',
    4: 'Grêmio',
    5: 'Fluminense',
    6: 'São Paulo',
    7: 'Palmeiras',
    8: 'Atlético Mineiro',
    9: 'Atlético PR',
    10: 'Corinthians',
    11: 'Vasco da Gama',
    12: 'Bahia',
    13: 'Atlético GO',
    14: 'Internacional', 
    15: 'Bragantino',
    16: 'Criciúma',
    17: 'Juventude',
    18: 'Cuiabá',
    19: 'Fortaleza'
}

In [ ]:
# Filtrar jogadores dos dois times específicos
time1 = times[1] # Flamengo
time2 = times[11] # Vasco da Gama

In [ ]:
# Filtrar os dados
data_time1 = data[data['Current Club'] == time1]
data_time2 = data[data['Current Club'] == time2]
data_filtered = pd.concat([data_time1, data_time2])

In [ ]:
X_filtered = data_filtered.select_dtypes(include=['number']).copy()
y_filtered = data_filtered['position'].apply(chance_of_goal)

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_filtered)

In [ ]:
model = Lasso(alpha=0.01, random_state=42)  
model.fit(X_scaled, y_filtered)
predictions = model.predict(X_scaled)

In [ ]:
# Aplicar a função sigmoide para transformar previsões em probabilidades
data_filtered['predicted_probability'] = sigmoid(predictions)

In [ ]:
best = data_filtered.sort_values(by='predicted_probability', ascending=False)

In [ ]:
best[['Current Club', 'full_name', 'predicted_probability']]


Ao analisar o resultado apresentado pelo modelo e realizar alguns testes manuais, foi possível ver que há uma certa imprecisão. Assim, foi calculado o MSE e o R² do modelo:

In [ ]:
# Calcular o MSE
mse = mean_squared_error(y_filtered, predictions)
 
# Calcular o MAE
mae = mean_absolute_error(y_filtered, predictions)

# Calcular o R²
r2 = r2_score(y_filtered, predictions)

# Exibir os resultados
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R²: {r2}')

Tendo em mente as métricas R², MSE e MAE, verificamos a importância das features utilizando SHAP.

In [ ]:
# Criar o explicador SHAP para o modelo
explainer = shap.LinearExplainer(model, X_filtered, feature_perturbation="interventional")

# Calcular os valores SHAP para o conjunto de dados filtrado
shap_values = explainer.shap_values(X_filtered)

# Visualizar um resumo gráfico dos valores SHAP para entender as principais variáveis
shap.summary_plot(shap_values, X_filtered, feature_names=X_filtered.columns)

Com isso, aplica-se o PCA para tentar melhorar as estatísticas do modelo.

In [ ]:
numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns
non_numeric_cols = data.select_dtypes(exclude=['float64', 'int64']).columns

In [ ]:
scaled_data = scaler.fit_transform(data[numeric_cols])

In [ ]:
pca = PCA(n_components=0.95)
pca_data = pca.fit_transform(scaled_data)

In [ ]:
# Criar um novo DataFrame com os componentes principais (PCA aplicado)
pca_columns = [f'PCA_{i+1}' for i in range(pca_data.shape[1])]
pca_df = pd.DataFrame(pca_data, columns=pca_columns)

print(pca_df.shape) # Verifica-se uma quantidade de 607 linhas e 28 colunas
print(data.shape) # Verifica-se uma quantidade de 607 linhas e 77 colunas

In [ ]:
final_df = pd.concat([df[non_numeric_cols].reset_index(drop=True), pca_df], axis=1)
final_df.to_csv('./Data/ppca.csv')

In [ ]:
df = pd.read_csv('./Data/ppca.csv')

In [ ]:
data_time1 = df[df['Current Club'] == time1]
data_time2 = df[df['Current Club'] == time2]
data_filtered = pd.concat([data_time1, data_time2])

In [ ]:
X_filtered = data_filtered.select_dtypes(include=['number']).copy()
y_filtered = data_filtered['position'].apply(chance_of_goal)

In [ ]:
model = Lasso(alpha=0.01, random_state=42)  
model.fit(X_filtered, y_filtered)
predictions = model.predict(X_filtered)

In [ ]:
# Aplicar a função sigmoide para transformar previsões em probabilidades
data_filtered['predicted_probability'] = sigmoid(predictions)

In [ ]:
best = data_filtered.sort_values(by='predicted_probability', ascending=False)
best[['Current Club', 'full_name', 'predicted_probability']]

In [ ]:
# Calcular o MSE
mse = mean_squared_error(y_filtered, predictions)

# Calcular o R²
r2 = r2_score(y_filtered, predictions)

# Exibir os resultados
print(f'MSE: {mse}')
print(f'R²: {r2}')

É perceptível uma pequena queda nas métricas, em relação ao modelo utilizando os dados puros (sem a aplicação do PCA), portanto, criamos outro gráfico para verificar a importância das features no modelo pelo SHAP.

In [ ]:
# Criar o explicador SHAP para o modelo
explainer = shap.LinearExplainer(model, X_filtered, feature_perturbation="interventional")

# Calcular os valores SHAP para o conjunto de dados filtrado
shap_values = explainer.shap_values(X_filtered)

# Visualizar um resumo gráfico dos valores SHAP para entender as principais variáveis
shap.summary_plot(shap_values, X_filtered, feature_names=X_filtered.columns)

Após analisar o gráfico SHAP gerado, verifica-se que a maioria das features (após a aplicação de PCA) estão influenciando negativamente no modelo.

In [ ]:
def apply_Lasso(alpha):
    model = Lasso(alpha=alpha, random_state=42)  
    model.fit(X_filtered, y_filtered)
    predictions = model.predict(X_filtered)
    data_filtered['predicted_probability'] = sigmoid(predictions)
    best = data_filtered.sort_values(by='predicted_probability', ascending=False)
    mae = mean_absolute_error(y_filtered, predictions)
    mse = mean_squared_error(y_filtered, predictions)
    r2 = r2_score(y_filtered, predictions)

    return [best[['Current Club', 'full_name', 'predicted_probability']], mse, r2, mae]

In [ ]:
model = apply_Lasso(0.02)
print(f'MSE: {model[1]}')
print(f'MAE: {model[3]}')
print(f'R²: {model[2]}')

In [ ]:
model = apply_Lasso(0.03)
print(f'MSE: {model[1]}')
print(f'MAE: {model[3]}')
print(f'R²: {model[2]}')

In [ ]:
model = apply_Lasso(0.04)
print(f'MSE: {model[1]}')
print(f'MAE: {model[3]}')
print(f'R²: {model[2]}')

Como obervado nas estatísticas, com o aumento do hiperparâmetro alpha do algoritmo LASSO, há uma aumenta no erro médio quadrático e uma diminuição no coeficiente R², portanto opta-se por manter o valor de alpha em 0,01. Além disso, foi notado que o algoritmo utilizando a tabela com PCA teve as estatísticas (MSE e R²) piores em relação ao algoritmo utilizando a tabela completa, porém em testes manuais foi notado uma melhora significativa. Então, decidimos utilizar o GridSearch como uma última tentativa de otimizar as métricas.

In [ ]:
param_grid = {'alpha': np.logspace(-4, 1, 50)}  # Testa valores de alpha entre 0.0001 e 10

# Instanciar o modelo LASSO
lasso = Lasso()

# Aplicar o GridSearchCV para otimizar o alpha
grid_search = GridSearchCV(lasso, param_grid, scoring='r2', cv=5)  # Usa R² como métrica e 5 folds para validação cruzada
grid_search.fit(X_scaled, y_filtered)

# Melhor valor de alpha
best_alpha = grid_search.best_params_['alpha']
print(f"Melhor valor de alpha: {best_alpha}")

Agora testamos o modelo com o alpha encontrado no Grid Search

In [ ]:
model = apply_Lasso(best_alpha)
print(f'MSE: {model[1]}')
print(f'MAE: {model[3]}')
print(f'R²: {model[2]}')

Com isso, podemos ver que as métricas sem PCA realmente não superaram as métricas sem PCA, porém empiricamente (com testes manuais) foi determinado que o modelo com o PCA funcionou melhor do que sem, assim, ficando claro que as métricas para esse modelo não são tão relevantes, pois se um zagueiro marcar gol, o modelo considerará isso como um erro, mesmo que isso realmente tenha acontecido.